# 2025-09-06

In [ ]:
# yield is like return, but it doesn't break out of the function!

import time

def countdown(n):
    while n > 0:
        yield n # this turns the function into a generator function
        n -= 1

#run_this = True
run_this = False
if run_this:
    c = countdown(10)
    for i in c:
        print(i)
        time.sleep(1)
    print("LIFTOFF")

# obvious issue with the above, using sleep, is that it doesn't actually fire once per second
# like delay in arduino, we need to replace it with the equivalent of millis


# __next__(self) is the method that when included in an object tells python that it can be iterated over 
# (used in loops and such)
# it's what the loop actually executes under the hood
c = countdown(3)
print(next(c)) # 3
print(next(c)) # 2
print(next(c)) # 1
try:
    print(next(c))
except Exception as e:
    print(f"Failed due to: {repr(e)}") # just printing `e` doesn't actually print "StopIteration", because that exception doesn't include a message, but it does have a __repr__ method (no __str__, though which is what happens when we run print(f"{e)") same as print(str(e))


# Background Timer with Interrupts
Like Arduino interrupts, we want a timer that runs independently and can interrupt other code execution.

In [ ]:
import time
import threading
from typing import Callable, Optional

class BackgroundTimer:
    """Arduino-style timer that runs in background and can interrupt other code"""
    
    def __init__(self, total_seconds: int, interval_seconds: int = 10):
        self.total_seconds = total_seconds
        self.interval_seconds = interval_seconds
        self.remaining = total_seconds
        self.running = False
        self.thread = None
        self.interrupt_flag = threading.Event()  # Like Arduino interrupt flag
        self.last_result = None
        
        # Custom thresholds and messages
        self.thresholds = {
            60: "Hurry up, half-time!",
            30: "30 seconds remaining!",
            10: "Final countdown!",
            5: "Almost done!",
            0: "TIME'S UP!"
        }
    
    def start(self):
        """Start the background timer"""
        if not self.running:
            self.running = True
            self.interrupt_flag.clear()
            self.thread = threading.Thread(target=self._timer_loop, daemon=True)
            self.thread.start()
            print(f"⏰ Timer started: {self.total_seconds}s countdown")
    
    def stop(self):
        """Stop the background timer"""
        self.running = False
        if self.thread:
            self.thread.join()
        print("⏹️ Timer stopped")
    
    def _timer_loop(self):
        """Internal timer loop - runs in background thread"""
        start_time = time.perf_counter()
        last_interval_print = 0
        
        while self.running and self.remaining > 0:
            elapsed = time.perf_counter() - start_time
            self.remaining = max(0, self.total_seconds - int(elapsed))
            
            # Print every interval
            current_interval = (self.total_seconds - self.remaining) // self.interval_seconds
            if current_interval > last_interval_print:
                print(f"\n⏱️ Time remaining: {self.remaining}s")
                last_interval_print = current_interval
                self.interrupt_flag.set()  # Signal interrupt
            
            # Check threshold messages
            if self.remaining in self.thresholds:
                print(f"\n🚨 {self.thresholds[self.remaining]}")
                self.interrupt_flag.set()  # Signal interrupt
                time.sleep(1)  # Prevent duplicate messages
            
            time.sleep(0.1)  # Check every 100ms
        
        if self.remaining <= 0:
            print(f"\n⏰ {self.thresholds[0]}")
            self.interrupt_flag.set()
        
        self.running = False
    
    def check_interrupt(self) -> bool:
        """Check if timer has triggered an interrupt (like Arduino interrupts)"""
        if self.interrupt_flag.is_set():
            self.interrupt_flag.clear()
            return True
        return False
    
    def is_time_up(self) -> bool:
        """Check if time is completely up"""
        return self.remaining <= 0 and not self.running

# Test the background timer
timer = BackgroundTimer(total_seconds=120, interval_seconds=10)
print("Timer created but not started yet...")

In [ ]:
def fibonacci_generator():
    """Infinite fibonacci generator - our 'main' code that runs continuously"""
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b

def run_with_timer_interrupts():
    """Main function that runs fibonacci calculation with timer interrupts"""
    
    # Start the timer
    timer = BackgroundTimer(total_seconds=15, interval_seconds=3)  # Shorter for demo
    timer.start()
    
    # Start our "main" computation
    fib_gen = fibonacci_generator()
    iteration_count = 0
    last_fib = 0
    
    print("🔢 Starting fibonacci calculation...")
    print("(Timer will interrupt every 3 seconds)")
    
    try:
        while not timer.is_time_up():
            # Do some work (calculate next fibonacci)
            last_fib = next(fib_gen)
            iteration_count += 1
            
            # Check for timer interrupt (like Arduino interrupt handling)
            if timer.check_interrupt():
                print(f"💡 INTERRUPT! Current fibonacci: {last_fib} (iteration {iteration_count})")
                print("   Continuing calculation...")
            
            # Small delay to make it visible (remove for real work)
            time.sleep(0.01)
            
            # Print progress occasionally
            if iteration_count % 500 == 0:
                print(f"📊 Progress: fibonacci #{iteration_count} = {last_fib}")
        
        print(f"\n🏁 Final result: fibonacci #{iteration_count} = {last_fib}")
        print("🛑 Main loop terminated - timer finished!")
        
    except KeyboardInterrupt:
        print(f"\n⚠️ Interrupted by user! Last result: {last_fib}")
    finally:
        timer.stop()

# Run the demo
run_with_timer_interrupts()

## Alternative: Signal-Style Interrupts
For even more Arduino-like behavior, here's a simpler approach using callbacks:

In [ ]:
import threading

class SimpleTimer:
    """Simple callback-based timer - even more like Arduino interrupts"""
    
    def __init__(self, total_seconds: int, callback_interval: int = 10):
        self.total_seconds = total_seconds
        self.callback_interval = callback_interval
        self.start_time = None
        self.running = False
        self.shared_data = {"last_result": None, "interrupt_count": 0, "time_up": False}
    
    def start(self, main_loop_function):
        """Start timer and run main loop with interrupts"""
        self.start_time = time.perf_counter()
        self.running = True
        self.shared_data["interrupt_count"] = 0
        self.shared_data["time_up"] = False
        
        print(f"🚀 Starting {self.total_seconds}s timer with {self.callback_interval}s interrupts")
        
        # Start background timer thread
        timer_thread = threading.Thread(target=self._timer_thread, daemon=True)
        timer_thread.start()
        
        # Run main loop
        try:
            main_loop_function(self.shared_data)
        finally:
            self.running = False
    
    def _timer_thread(self):
        """Background thread that triggers interrupts"""
        last_interrupt = 0
        
        while self.running:
            elapsed = time.perf_counter() - self.start_time
            remaining = max(0, self.total_seconds - elapsed)
            
            # Check for interval interrupts
            current_interval = int(elapsed // self.callback_interval)
            if current_interval > last_interrupt:
                last_interrupt = current_interval
                self.shared_data["interrupt_count"] += 1
                
                # This is the "interrupt" - print status
                print(f"\n⚡ TIMER INTERRUPT #{self.shared_data['interrupt_count']}")
                print(f"   Time remaining: {remaining:.1f}s")
                print(f"   Last result: {self.shared_data['last_result']}")
                
                # Threshold messages
                if remaining <= 10:
                    print("   🔥 FINAL COUNTDOWN!")
                elif remaining <= 30:
                    print("   ⚠️ 30 seconds or less!")
                elif remaining <= 60:
                    print("   ⏰ Half time!")
            
            # Check if time is up
            if remaining <= 0:
                print(f"\n🏁 TIME'S UP! Final result: {self.shared_data['last_result']}")
                self.shared_data["time_up"] = True  # Signal main loop to stop
                self.running = False
                break
                
            time.sleep(0.1)

def my_main_loop(shared_data):
    """Main computation loop - calculates fibonacci while timer interrupts"""
    fib_gen = fibonacci_generator()
    count = 0
    
    # Main loop now checks if time is up
    while not shared_data["time_up"]:
        # Do the main work
        result = next(fib_gen)
        count += 1
        
        # Update shared data (like global variables in Arduino)
        shared_data["last_result"] = f"fib[{count}] = {result}"
        
        # Print progress occasionally
        if count % 5000 == 0:
            print(f"📈 Computing... {shared_data['last_result']}")
        
        # Small delay to make demo visible
        time.sleep(0.001)
    
    print("🛑 Main loop stopped - timer finished!")

# Demo the simple timer
timer = SimpleTimer(total_seconds=20, callback_interval=3)
print("Created timer. Call timer.start(my_main_loop) to begin!")

In [ ]:
# Test the fixed SimpleTimer - should stop after 10 seconds
test_timer = SimpleTimer(total_seconds=10, callback_interval=2)
print("🧪 Testing SimpleTimer with 10s countdown, 2s intervals...")
test_timer.start(my_main_loop)

## Arduino Comparison

This Python approach mimics Arduino timer interrupts:

**Arduino:**
```cpp
void setup() {
    // Setup timer interrupt every 1 second
    Timer1.initialize(1000000); // 1 second
    Timer1.attachInterrupt(timerISR);
}

void timerISR() {
    // Interrupt service routine
    Serial.println("Timer interrupt!");
    timeRemaining--;
}

void loop() {
    // Main code runs continuously
    fibonacci_calculation();
}
```

**Python equivalent:**
- `threading.Thread` = Arduino timer interrupt
- `shared_data` = global variables accessible in ISR
- `check_interrupt()` = checking interrupt flags
- Main loop = Arduino `loop()` function

The key insight: **The timer runs independently and can interrupt/communicate with your main code**, just like hardware interrupts!

## Summary: Background Timer Patterns

You now have **two main approaches** for background timers that interrupt other code:

### 1. **Event Flag Pattern** (`BackgroundTimer`)
- Main code checks `timer.check_interrupt()` periodically
- More control over when interrupts are handled
- Like Arduino polling interrupt flags

### 2. **Callback Pattern** (`SimpleTimer`) 
- Timer directly prints/executes interrupt code
- Main code just runs, timer handles itself
- More like true hardware interrupts

**Key Concepts:**
- `threading.Thread(daemon=True)` = background timer
- `threading.Event()` = interrupt flag signaling
- Shared data structures = global variables in Arduino
- `time.perf_counter()` = precise timing like `millis()`

Both patterns let your main code run continuously while the timer operates independently in the background!

## ✅ Fixed: Proper Timer Termination

**The Problem:** The main loop was running `while True:` with no exit condition.

**The Solution:** Added shared state communication:

1. **Timer sets flag**: `shared_data["time_up"] = True` when timer expires
2. **Main loop checks flag**: `while not shared_data["time_up"]:` instead of `while True:`
3. **Clean termination**: Main loop exits when timer finishes

This is exactly like Arduino global variables that both the main loop and interrupt service routine can access!

**Arduino equivalent:**
```cpp
volatile bool timeUp = false;  // Global variable

void timerISR() {
    if (timeRemaining <= 0) {
        timeUp = true;  // Set flag in interrupt
    }
}

void loop() {
    while (!timeUp) {  // Check flag in main loop
        // Do work
        fibonacci_calculation();
    }
    Serial.println("Timer finished!");
}
```

# Advanced: Proper Shared Data Access

You're absolutely right! The current `shared_data` is only accessible to the function passed to `start()`. Let's fix this with proper shared state management.

In [10]:
import threading
import time
from dataclasses import dataclass
from typing import Any, Dict

def fibonacci_generator():
    """Infinite fibonacci generator"""
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b

@dataclass
class SharedState:
    """Thread-safe shared state using locks"""
    def __init__(self):
        self._data = {
            "last_result": None,
            "interrupt_count": 0,
            "time_up": False,
            "user_message": "",
            "computation_speed": 0.001
        }
        self._lock = threading.Lock()  # Protects concurrent access
    
    def get(self, key: str) -> Any:
        """Thread-safe read"""
        with self._lock:
            return self._data.get(key)
    
    def set(self, key: str, value: Any) -> None:
        """Thread-safe write"""
        with self._lock:
            self._data[key] = value
    
    def update_multiple(self, updates: Dict[str, Any]) -> None:
        """Thread-safe batch update"""
        with self._lock:
            self._data.update(updates)
    
    def get_all(self) -> Dict[str, Any]:
        """Thread-safe read of all data"""
        with self._lock:
            return self._data.copy()

class GlobalTimer:
    """Timer with globally accessible shared state"""
    
    def __init__(self, total_seconds: int, callback_interval: int = 3):
        self.total_seconds = total_seconds
        self.callback_interval = callback_interval
        self.start_time = None
        self.running = False
        self.shared_state = SharedState()  # Now accessible from anywhere
        
    def start(self):
        """Start timer - no function parameter needed"""
        self.start_time = time.perf_counter()
        self.running = True
        self.shared_state.update_multiple({
            "interrupt_count": 0,
            "time_up": False
        })
        
        print(f"🚀 Starting {self.total_seconds}s timer with {self.callback_interval}s interrupts")
        
        # Start background timer thread
        timer_thread = threading.Thread(target=self._timer_thread, daemon=True)
        timer_thread.start()
    
    def stop(self):
        """Stop the timer"""
        self.running = False
    
    def _timer_thread(self):
        """Background thread that triggers interrupts"""
        last_interrupt = 0
        
        while self.running:
            elapsed = time.perf_counter() - self.start_time
            remaining = max(0, self.total_seconds - elapsed)
            
            # Check for interval interrupts
            current_interval = int(elapsed // self.callback_interval)
            if current_interval > last_interrupt:
                last_interrupt = current_interval
                
                # Thread-safe update
                interrupt_count = self.shared_state.get("interrupt_count") + 1
                self.shared_state.update_multiple({
                    "interrupt_count": interrupt_count
                })
                
                # This is the "interrupt"
                print(f"\n⚡ TIMER INTERRUPT #{interrupt_count}")
                print(f"   Time remaining: {remaining:.1f}s")
                print(f"   Last result: {self.shared_state.get('last_result')}")
                print(f"   User message: {self.shared_state.get('user_message')}")
                
                # Threshold messages
                if remaining <= 5:
                    print("   🔥 FINAL COUNTDOWN!")
                elif remaining <= 10:
                    print("   ⚠️ 10 seconds or less!")
            
            # Check if time is up
            if remaining <= 0:
                print(f"\n🏁 TIME'S UP! Final result: {self.shared_state.get('last_result')}")
                self.shared_state.set("time_up", True)
                self.running = False
                break
                
            time.sleep(0.1)

# Global timer instance - accessible from anywhere
global_timer = GlobalTimer(total_seconds=15, callback_interval=3)
print("Created global timer with shared state accessible from any function!")

Created global timer with shared state accessible from any function!


In [ ]:
# Now multiple functions can access the shared state
def main_computation():
    """Main fibonacci computation - runs in main thread"""
    fib_gen = fibonacci_generator()
    count = 0
    
    while not global_timer.shared_state.get("time_up"):
        result = next(fib_gen)
        count += 1
        
        # Update shared state
        global_timer.shared_state.set("last_result", f"fib[{count}] = {result}")
        
        # Print progress occasionally
        if count % 2000 == 0:
            print(f"📈 Main: {global_timer.shared_state.get('last_result')}")
        
        # Use the configurable speed
        speed = global_timer.shared_state.get("computation_speed")
        time.sleep(speed)
    
    print("🛑 Main computation stopped!")

def user_input_monitor():
    """Simulated user input - runs in separate thread"""
    messages = [
        "Keep going!",
        "Halfway there!",
        "Almost done!",
        "Final push!"
    ]
    
    for i, msg in enumerate(messages):
        time.sleep(4)  # Wait 4 seconds between messages
        if global_timer.shared_state.get("time_up"):
            break
            
        # Update shared state from another thread
        global_timer.shared_state.set("user_message", msg)
        print(f"👤 User says: {msg}")
        
        # Maybe speed up computation near the end
        if i >= 2:
            global_timer.shared_state.set("computation_speed", 0.0005)
            print("   ⚡ Speeding up computation!")

def performance_monitor():
    """Monitor performance - runs in separate thread"""
    last_count = 0
    
    while not global_timer.shared_state.get("time_up"):
        time.sleep(2)
        
        # Read current state
        current_result = global_timer.shared_state.get("last_result") or "fib[0] = 0"
        
        # Extract count from result string
        try:
            current_count = int(current_result.split('[')[1].split(']')[0])
            rate = (current_count - last_count) / 2  # calculations per second
            print(f"📊 Performance: {rate:.0f} calculations/sec")
            last_count = current_count
        except:
            pass

def run_threaded_example():
    """Run example with multiple threads accessing shared state"""
    print("🏁 Starting threaded example with multiple functions accessing shared state...")
    
    # Start the timer
    global_timer.start()
    
    # Start monitoring threads
    user_thread = threading.Thread(target=user_input_monitor, daemon=True)
    perf_thread = threading.Thread(target=performance_monitor, daemon=True)
    
    user_thread.start()
    perf_thread.start()
    
    # Run main computation in current thread
    try:
        main_computation()
    except KeyboardInterrupt:
        print("\n⚠️ Interrupted by user!")
    finally:
        global_timer.stop()

# Run the example
run_threaded_example()

🏁 Starting threaded example with multiple functions accessing shared state...
🚀 Starting 15s timer with 3s interrupts
📊 Performance: 853 calculations/sec
📊 Performance: 853 calculations/sec
📈 Main: fib[2000] = 2611005926183501768338670946829097324475555189114843467397273230483773870037923307730410719313972291638157639230613843870597997481070930648667960025707364078851859017098672504986584144842548768373271309551281830431960537091677315014266625027123872238011234749984205478230617988978500613170516952885123444971471854671812569739975450866912490650853945622130138277040986146312325044424769652148982077548213909414076005501
📈 Main: fib[2000] = 261100592618350176833867094682909732447555518911484346739727323048377387003792330773041071931397229163815763923061384387059799748107093064866796002570736407885185901709867250498658414484254876837327130955128183043196053709167731501426662502712387223801123474998420547823061798897850061317051695288512344497147185467181256973997545086691249065085394562

📊 Performance: 677 calculations/sec


## AsyncIO Version: Coroutines Instead of Threads

Now let's see the same concept using asyncio - no locks needed because it's single-threaded cooperative multitasking!

In [12]:
import asyncio
from dataclasses import dataclass
import time

class AsyncSharedState:
    """Simple shared state for asyncio - no locks needed!"""
    def __init__(self):
        self.data = {
            "last_result": None,
            "interrupt_count": 0,
            "time_up": False,
            "user_message": "",
            "computation_speed": 0.01,  # Slower for demo
            "start_time": None
        }
    
    def get(self, key: str):
        return self.data.get(key)
    
    def set(self, key: str, value):
        self.data[key] = value
    
    def update(self, updates: dict):
        self.data.update(updates)

class AsyncTimer:
    """AsyncIO-based timer using coroutines"""
    
    def __init__(self, total_seconds: int, callback_interval: int = 3):
        self.total_seconds = total_seconds
        self.callback_interval = callback_interval
        self.shared_state = AsyncSharedState()
        self.running = False
    
    async def start_timer(self):
        """Timer coroutine - runs concurrently with other tasks"""
        self.running = True
        self.shared_state.set("start_time", time.perf_counter())
        self.shared_state.update({
            "interrupt_count": 0,
            "time_up": False
        })
        
        print(f"🚀 Starting async {self.total_seconds}s timer with {self.callback_interval}s interrupts")
        
        last_interrupt = 0
        
        while self.running:
            start_time = self.shared_state.get("start_time")
            elapsed = time.perf_counter() - start_time
            remaining = max(0, self.total_seconds - elapsed)
            
            # Check for interval interrupts
            current_interval = int(elapsed // self.callback_interval)
            if current_interval > last_interrupt:
                last_interrupt = current_interval
                interrupt_count = self.shared_state.get("interrupt_count") + 1
                self.shared_state.set("interrupt_count", interrupt_count)
                
                # This is the "interrupt"
                print(f"\n⚡ ASYNC INTERRUPT #{interrupt_count}")
                print(f"   Time remaining: {remaining:.1f}s")
                print(f"   Last result: {self.shared_state.get('last_result')}")
                print(f"   User message: {self.shared_state.get('user_message')}")
                
                # Threshold messages
                if remaining <= 5:
                    print("   🔥 FINAL COUNTDOWN!")
                elif remaining <= 10:
                    print("   ⚠️ 10 seconds or less!")
            
            # Check if time is up
            if remaining <= 0:
                print(f"\n🏁 ASYNC TIME'S UP! Final: {self.shared_state.get('last_result')}")
                self.shared_state.set("time_up", True)
                self.running = False
                break
            
            # Yield control to other coroutines
            await asyncio.sleep(0.1)
    
    def stop(self):
        self.running = False

# Global async timer
async_timer = AsyncTimer(total_seconds=15, callback_interval=3)
print("Created async timer!")

Created async timer!


In [13]:
async def async_main_computation():
    """Main fibonacci computation - async coroutine"""
    fib_gen = fibonacci_generator()
    count = 0
    
    while not async_timer.shared_state.get("time_up"):
        result = next(fib_gen)
        count += 1
        
        # Update shared state (no locks needed!)
        async_timer.shared_state.set("last_result", f"fib[{count}] = {result}")
        
        # Print progress occasionally
        if count % 500 == 0:
            print(f"📈 Async Main: {async_timer.shared_state.get('last_result')}")
        
        # Yield control to other coroutines
        speed = async_timer.shared_state.get("computation_speed")
        await asyncio.sleep(speed)
    
    print("🛑 Async main computation stopped!")

async def async_user_input():
    """Simulated user input - async coroutine"""
    messages = [
        "Keep going async!",
        "Halfway there!",
        "Async is awesome!",
        "Final push!"
    ]
    
    for i, msg in enumerate(messages):
        await asyncio.sleep(4)  # Wait 4 seconds
        if async_timer.shared_state.get("time_up"):
            break
            
        # Update shared state
        async_timer.shared_state.set("user_message", msg)
        print(f"👤 Async User: {msg}")
        
        # Speed up near the end
        if i >= 2:
            async_timer.shared_state.set("computation_speed", 0.005)
            print("   ⚡ Async speedup!")

async def async_performance_monitor():
    """Monitor performance - async coroutine"""
    last_count = 0
    
    while not async_timer.shared_state.get("time_up"):
        await asyncio.sleep(2)
        
        current_result = async_timer.shared_state.get("last_result") or "fib[0] = 0"
        
        try:
            current_count = int(current_result.split('[')[1].split(']')[0])
            rate = (current_count - last_count) / 2
            print(f"📊 Async Performance: {rate:.0f} calc/sec")
            last_count = current_count
        except:
            pass

async def run_async_example():
    """Run all async coroutines concurrently"""
    print("🏁 Starting async example with concurrent coroutines...")
    
    # Run all coroutines concurrently
    await asyncio.gather(
        async_timer.start_timer(),
        async_main_computation(),
        async_user_input(),
        async_performance_monitor()
    )

# Note: asyncio.run() doesn't work in notebooks because they already have an event loop
# Instead, use await in notebook:
print("📝 Note: Run 'await run_async_example()' in a separate cell to test asyncio version")
print("    (asyncio.run() doesn't work in notebooks)")

# For testing outside notebooks, you would use:
# asyncio.run(run_async_example())

📝 Note: Run 'await run_async_example()' in a separate cell to test asyncio version
    (asyncio.run() doesn't work in notebooks)


In [14]:
# Run the async example (works in notebooks)
await run_async_example()

🏁 Starting async example with concurrent coroutines...
🚀 Starting async 15s timer with 3s interrupts
📊 Async Performance: 96 calc/sec
📊 Async Performance: 96 calc/sec

⚡ ASYNC INTERRUPT #1
   Time remaining: 12.0s
   Last result: fib[292] = 2923602405716568564338475449381171413803636207598822186175234
   User message: 

⚡ ASYNC INTERRUPT #1
   Time remaining: 12.0s
   Last result: fib[292] = 2923602405716568564338475449381171413803636207598822186175234
   User message: 
👤 Async User: Keep going async!
📊 Async Performance: 96 calc/sec
👤 Async User: Keep going async!
📊 Async Performance: 96 calc/sec
📈 Async Main: fib[500] = 86168291600238450732788312165664788095941068326060883324529903470149056115823592713458328176574447204501
📈 Async Main: fib[500] = 86168291600238450732788312165664788095941068326060883324529903470149056115823592713458328176574447204501
📊 Async Performance: 96 calc/sec

⚡ ASYNC INTERRUPT #2
   Time remaining: 8.9s
   Last result: fib[581] = 73003947283373540362408137551

## Threading vs AsyncIO vs Multiprocessing Comparison

### **⚠️ Important: Python's GIL (Global Interpreter Lock)**

Python's GIL prevents true parallelism for CPU-bound tasks in threading!

### **Threading Approach** 🧵
```python
# Shared state needs locks for thread safety
class SharedState:
    def __init__(self):
        self._lock = threading.Lock()
    
    def set(self, key, value):
        with self._lock:  # Critical section
            self._data[key] = value

# Multiple threads - but GIL limits CPU parallelism!
threading.Thread(target=function1).start()
threading.Thread(target=function2).start()
```

**Pros:**
- Good for I/O-bound tasks (file, network operations)
- Can release GIL during I/O waits
- Shared memory between threads

**Cons:**
- **NO true CPU parallelism** due to GIL
- Need locks to prevent race conditions
- More complex debugging

### **AsyncIO Approach** ⚡
```python
# No locks needed - single-threaded
class AsyncSharedState:
    def set(self, key, value):
        self.data[key] = value  # No locks needed!

# Cooperative multitasking
await asyncio.gather(
    coroutine1(),
    coroutine2()
)
```

**Pros:**
- No race conditions (single-threaded)
- No locks needed
- Great for I/O-bound tasks
- Lower memory overhead than threading

**Cons:**
- No parallelism at all (single-threaded)
- One blocking operation blocks everything
- Must use `await` to yield control

### **Multiprocessing Approach** 🚀 (True Parallelism!)
```python
import multiprocessing as mp

# Each process has separate memory space
def worker_process(shared_queue, shared_value):
    # True parallelism - separate Python interpreter!
    result = cpu_intensive_work()
    shared_queue.put(result)

# Multiple processes - bypasses GIL completely
processes = []
for i in range(mp.cpu_count()):
    p = mp.Process(target=worker_process, args=(queue, value))
    p.start()
    processes.append(p)
```

**Pros:**
- **TRUE parallelism** across CPU cores
- Bypasses GIL completely
- Separate memory spaces (crash isolation)

**Cons:**
- Higher memory overhead
- Complex inter-process communication
- No shared memory (must use queues, pipes, etc.)

### **When to Use What?**

- **Threading**: I/O-bound tasks (file operations, API calls, database queries)
- **AsyncIO**: Many concurrent I/O operations, single-threaded efficiency  
- **Multiprocessing**: CPU-intensive work that needs true parallelism

In [16]:
import multiprocessing as mp
import queue
import time

def cpu_intensive_fibonacci(n, result_queue, process_id):
    """CPU-intensive fibonacci calculation in separate process"""
    def fib(x):
        if x <= 1:
            return x
        return fib(x-1) + fib(x-2)
    
    print(f"🔥 Process {process_id}: Starting fibonacci({n})")
    start_time = time.perf_counter()
    
    result = fib(n)
    
    elapsed = time.perf_counter() - start_time
    result_queue.put({
        'process_id': process_id,
        'input': n,
        'result': result,
        'time': elapsed
    })
    print(f"✅ Process {process_id}: fibonacci({n}) = {result} (took {elapsed:.2f}s)")

def multiprocessing_timer_example():
    """Example showing true parallelism with multiprocessing"""
    print("🚀 Demonstrating TRUE parallelism with multiprocessing...")
    print(f"Available CPU cores: {mp.cpu_count()}")
    
    # Create shared queue for results
    result_queue = mp.Queue()
    
    # Different fibonacci numbers to calculate
    tasks = [35, 36, 37, 38]  # These take progressively longer
    
    # Start timer
    overall_start = time.perf_counter()
    
    # Create and start processes
    processes = []
    for i, n in enumerate(tasks):
        p = mp.Process(target=cpu_intensive_fibonacci, args=(n, result_queue, i+1))
        p.start()
        processes.append(p)
        print(f"📋 Started process {i+1} for fibonacci({n})")
    
    # Collect results as they complete
    results = []
    for _ in range(len(tasks)):
        try:
            result = result_queue.get(timeout=30)  # 30 second timeout
            results.append(result)
            print(f"📊 Received result from process {result['process_id']}")
        except queue.Empty:
            print("⚠️ Timeout waiting for result")
    
    # Wait for all processes to complete
    for p in processes:
        p.join()
    
    overall_time = time.perf_counter() - overall_start
    
    print(f"\n🏁 All processes completed in {overall_time:.2f}s")
    print("📈 Results summary:")
    for result in sorted(results, key=lambda x: x['process_id']):
        print(f"   Process {result['process_id']}: fib({result['input']}) = {result['result']} ({result['time']:.2f}s)")
    
    # Calculate what it would take sequentially
    total_sequential = sum(r['time'] for r in results)
    speedup = total_sequential / overall_time
    print(f"\n⚡ Speedup: {speedup:.1f}x faster than sequential execution")
    print(f"   (Sequential would take ~{total_sequential:.1f}s)")

# Note: This will only work if run as a script due to multiprocessing requirements
print("📝 Note: Multiprocessing examples work best when run as scripts")
print("    In notebooks, you may see limited parallelism due to execution model")

# Uncomment to run (may not work in all notebook environments):
# if __name__ == '__main__':
#     multiprocessing_timer_example()

📝 Note: Multiprocessing examples work best when run as scripts
    In notebooks, you may see limited parallelism due to execution model


In [17]:
# Simple demonstration of GIL limitations
import threading
import time

def cpu_bound_task(name, duration):
    """CPU-intensive task that shows GIL limitations"""
    start = time.perf_counter()
    
    # CPU-intensive work (calculating primes)
    count = 0
    for i in range(1000000):
        for j in range(2, int(i**0.5) + 1):
            if i % j == 0:
                break
        else:
            count += 1
            if count >= 100:  # Limit to prevent long execution
                break
    
    elapsed = time.perf_counter() - start
    print(f"🧵 Thread {name}: Found {count} primes in {elapsed:.3f}s")

def demonstrate_gil_limitation():
    """Show that threading doesn't provide CPU parallelism due to GIL"""
    print("🔬 Demonstrating GIL limitation with CPU-bound tasks...")
    
    # Sequential execution
    print("\n📊 Sequential execution:")
    start = time.perf_counter()
    cpu_bound_task("Sequential-1", 2)
    cpu_bound_task("Sequential-2", 2)
    sequential_time = time.perf_counter() - start
    print(f"Total sequential time: {sequential_time:.3f}s")
    
    # Threaded execution (should be similar time due to GIL!)
    print("\n🧵 Threaded execution:")
    start = time.perf_counter()
    threads = [
        threading.Thread(target=cpu_bound_task, args=("Thread-1", 2)),
        threading.Thread(target=cpu_bound_task, args=("Thread-2", 2))
    ]
    
    for t in threads:
        t.start()
    
    for t in threads:
        t.join()
    
    threaded_time = time.perf_counter() - start
    print(f"Total threaded time: {threaded_time:.3f}s")
    
    print(f"\n📈 Analysis:")
    print(f"   Sequential: {sequential_time:.3f}s")
    print(f"   Threaded:   {threaded_time:.3f}s")
    
    if threaded_time < sequential_time * 0.7:
        print("   ✅ Significant speedup - unexpected for CPU-bound tasks!")
    else:
        print("   ⚠️ No significant speedup - this demonstrates the GIL!")
        print("   🔍 Threads can't run Python code in parallel due to GIL")

# Run the demonstration
demonstrate_gil_limitation()

🔬 Demonstrating GIL limitation with CPU-bound tasks...

📊 Sequential execution:
🧵 Thread Sequential-1: Found 100 primes in 0.000s
🧵 Thread Sequential-2: Found 100 primes in 0.002s
Total sequential time: 0.002s

🧵 Threaded execution:
🧵 Thread Thread-1: Found 100 primes in 0.000s
🧵 Thread Thread-2: Found 100 primes in 0.000s
Total threaded time: 0.007s

📈 Analysis:
   Sequential: 0.002s
   Threaded:   0.007s
   ⚠️ No significant speedup - this demonstrates the GIL!
   🔍 Threads can't run Python code in parallel due to GIL


## ✅ Answer to Your Questions (CORRECTED!)

### **1. Shared Data Access Beyond One Function**

**Problem**: Original `SimpleTimer` only passed `shared_data` to one function.

**Solution**: Make shared state globally accessible:

```python
# Instead of this:
timer.start(my_function)  # Only my_function gets shared_data

# Do this:
class GlobalTimer:
    def __init__(self):
        self.shared_state = SharedState()  # Accessible from anywhere

# Now ANY function can access it:
def function1():
    global_timer.shared_state.set("key", "value")

def function2():
    value = global_timer.shared_state.get("key")
```

### **2. Threading Locks - Yes, You Need Them!**

**Threading version**:
```python
class SharedState:
    def __init__(self):
        self._lock = threading.Lock()  # CRITICAL!
    
    def set(self, key, value):
        with self._lock:  # Prevents race conditions
            self._data[key] = value
```

**Why?** Multiple threads can access data simultaneously, causing race conditions.

### **3. AsyncIO - No Locks Needed!**

**AsyncIO version**:
```python
class AsyncSharedState:
    def set(self, key, value):
        self.data[key] = value  # No locks needed!
```

**Why?** Single-threaded cooperative multitasking - only one coroutine runs at a time.

### **⚠️ IMPORTANT CORRECTION: Python's GIL**

**You are 100% correct!** I made an error about threading and parallelism:

- **Threading in Python**: NOT true parallelism due to GIL
- **GIL (Global Interpreter Lock)**: Only one thread can execute Python code at a time
- **Threading is good for**: I/O-bound tasks (network, file operations)
- **For TRUE CPU parallelism**: Use `multiprocessing` module

### **Key Takeaways (CORRECTED)**

- **Threading**: Good for I/O-bound tasks, shared memory, but GIL limits CPU parallelism
- **AsyncIO**: Single-threaded, great for many concurrent I/O operations
- **Multiprocessing**: TRUE parallelism across CPU cores, bypasses GIL completely
- **Global access**: Make shared state a class attribute, not function parameter